In [2]:
# Import packages

import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import gc #Garbage Collector interface
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb

In [7]:
# Import datasets
data_df = pd.read_excel('/Users/Stylewsxcde991/Desktop/金融科技_文字探勘與機器學習/金融科技Final_project/2019年看門狗資訊事件整理.xlsx',index_col=0)

In [8]:
print("data_df - rows:",data_df.shape[0]," columns:", data_df.shape[1])

data_df - rows: 4040  columns: 7


In [9]:
data_df.head()

,公司簡稱,事件日,TCRI(年/月),事件強度,大事件類別,小事件類別,事件內容
個股代號,,,,,,,
1218,泰山,20190101,6(2018/09),0,M_經營層,MT06_高管異動,發言人林俐婉內部調動，由江巍峰接任。。
1503,士電,20190101,4(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管林志強內部調動，由莊文清接任。。
1504,東元,20190101,4(2018/09),0,M_經營層,MT06_高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。
1709,和益,20190101,5(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。
1721,三晃,20190101,7(2018/09),0,M_經營層,MT06_高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。


In [11]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4040 entries, 1218 to 6486
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   公司簡稱       3805 non-null   object
 1   事件日        4040 non-null   int64 
 2   TCRI(年/月)  4040 non-null   object
 3   事件強度       4040 non-null   int64 
 4   大事件類別      4040 non-null   object
 5   小事件類別      4040 non-null   object
 6   事件內容       4040 non-null   object
dtypes: int64(2), object(5)
memory usage: 252.5+ KB


In [13]:
# The proportion of each target class

data_df["事件強度"].value_counts()

-1    2386
 0    1242
 1     299
-2      80
-3      32
 3       1
Name: 事件強度, dtype: int64

In [15]:
import jieba.posseg as pseg

text = '我是李孟，在東京工作的數據科學家'
words = pseg.cut(text)
[word for word in words]

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xk/80wyspyn4cz5bfm6xr2yw1380000gn/T/jieba.cache
Loading model cost 1.198 seconds.
Prefix dict has been built successfully.


[pair('我', 'r'),
 pair('是', 'v'),
 pair('李孟', 'nr'),
 pair('，', 'x'),
 pair('在', 'p'),
 pair('東京', 'ns'),
 pair('工作', 'vn'),
 pair('的', 'uj'),
 pair('數據', 'n'),
 pair('科學家', 'n')]

In [16]:
def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([
        word for word, flag in words if flag != 'x'])

In [17]:
jieba_tokenizer(text)

'我 是 李孟 在 東京 工作 的 數據 科學家'

In [42]:
data_df['content_tokenized'] = data_df.iloc[:10, 6].apply(jieba_tokenizer)

In [43]:
data_df

,公司簡稱,事件日,TCRI(年/月),事件強度,大事件類別,小事件類別,事件內容,content_tokenized
個股代號,,,,,,,,
1218,泰山,20190101,6(2018/09),0,M_經營層,MT06_高管異動,發言人林俐婉內部調動，由江巍峰接任。。,發言 人 林俐婉 內部 調動 由 江巍峰 接任
1503,士電,20190101,4(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管林志強內部調動，由莊文清接任。。,內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
1504,東元,20190101,4(2018/09),0,M_經營層,MT06_高管異動,會計主管藍俊雄內部調動，由林鴻名接任。。,會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
1709,和益,20190101,5(2018/09),0,M_經營層,MT06_高管異動,內部稽核主管游本詮內部調動，由曾筱茜接任。。,內部 稽核 主管 游本 詮內部 調動 由 曾筱茜 接任
1721,三晃,20190101,7(2018/09),0,M_經營層,MT06_高管異動,財務經理洪廷宜內部調動，由王婷渝接任。。,財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
...,...,...,...,...,...,...,...,...
4726,永昕,20190331,7(2018/09),-1,M_經營層,MT06_高管異動,財務經理陳炯祥離職。。發言人陳炯祥離職。。會計主管陳炯祥離職。。,NaN
4960,誠美材,20190331,D(2019/03),-2,A_會計/財報分析,AF09_非無保留意見,誠美材 2018年度會計師出具保留意見加繼續經營有關之重大不確定性段落之查核報告：誠美材截至...,NaN
5481,新華,20190331,8(2018/09),-1,M_經營層,MT06_高管異動,總經理陳聖中離職。。,NaN


In [37]:
data_df['content_tokenized'] = data_df.loc[:, '事件內容'].apply(jieba_tokenizer)

KeyboardInterrupt: 

In [ ]:
data_df['big_tokenized'] = data_df.loc[:, '大事件類別'].apply(jieba_tokenizer)
data_df['small_tokenized'] = data_df.loc[:, '小事件類別'].apply(jieba_tokenizer)

In [ ]:
import keras
MAX_NUM_WORDS = 10000
tokenizer = keras .preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)